In [22]:
import prophet as pt
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# Import postgres python adapter
import psycopg2
import logging
import os
import datetime
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


True

In [25]:
host = os.getenv("HOST")
port = os.getenv("PORT")
database = os.getenv("DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("PASSWORD")

In [26]:
# Read data from postgres
def read_data():
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cur = conn.cursor()
    cur.execute("SELECT * FROM aggregated_demand")
    data = cur.fetchall()
    cur.close()
    conn.close()
    return data

In [27]:
# Convert data to dataframe
def convert_to_dataframe(data):
    df = pd.DataFrame(data, columns=['ds', 'state', 'frequency', 'demand'])
    df['ds'] = pd.to_datetime(df['ds'])
    return df


In [28]:
data = read_data()

In [29]:
data = convert_to_dataframe(data)

In [31]:
data.head(50)

,ds,state,frequency,demand
0,2021-01-01 01:15:00,DD,50.0365517241379,252.241379310345
1,2021-01-01 01:15:00,Chattishgarh,50.0365517241379,3235.75862068966
2,2021-01-01 01:15:00,Maharashtra,50.0365517241379,18951.1379310345
3,2021-01-01 01:15:00,Goa,50.0365517241379,403.758620689655
4,2021-01-01 01:15:00,Gujrat,50.0365517241379,13756.724137931
5,2021-01-01 01:15:00,Madhya Pradesh,50.0365517241379,12378.275862069
6,2021-01-01 01:15:00,DNH,50.0365517241379,722
7,2021-01-01 01:30:00,Madhya Pradesh,50.0237931034483,12375.6551724138
8,2021-01-01 01:30:00,Gujrat,50.0237931034483,13706.1724137931
9,2021-01-01 01:30:00,Maharashtra,50.0237931034483,18829.3793103448


In [32]:
# Create a model for each state
states = data['state'].unique()
models = {}

for state in states:
    df = data[data['state'] == state]
    model = pt.Prophet()
    df = df[['ds', 'demand']]
    df.columns = ['ds', 'y']
    model.fit(df)
    models[state] = model


14:20:02 - cmdstanpy - INFO - Chain [1] start processing
14:22:15 - cmdstanpy - INFO - Chain [1] done processing
14:22:19 - cmdstanpy - INFO - Chain [1] start processing
14:24:04 - cmdstanpy - INFO - Chain [1] done processing
14:24:07 - cmdstanpy - INFO - Chain [1] start processing
14:24:19 - cmdstanpy - INFO - Chain [1] done processing
14:24:22 - cmdstanpy - INFO - Chain [1] start processing
14:25:20 - cmdstanpy - INFO - Chain [1] done processing
14:25:23 - cmdstanpy - INFO - Chain [1] start processing
14:28:15 - cmdstanpy - INFO - Chain [1] done processing
14:28:20 - cmdstanpy - INFO - Chain [1] start processing
14:30:36 - cmdstanpy - INFO - Chain [1] done processing
14:30:40 - cmdstanpy - INFO - Chain [1] start processing
14:33:09 - cmdstanpy - INFO - Chain [1] done processing


In [33]:
# create forecast for each state
forecasts = {}
for state in states:
    model = models[state]
    future = model.make_future_dataframe(periods=4*24*3)
    forecast = model.predict(future)
    forecasts[state] = forecast

In [9]:
# Plot forecast for each state
for state in states:
    forecast = forecasts[state]
    fig = plot_plotly(model, forecast)
    fig.update_layout(
        title=f"Forecast for {state}",
        xaxis_title="Date",
        yaxis_title="Demand (MW)",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
!pip install - -upgrade nbformat



Usage:   
  /usr/local/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/local/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/local/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/local/bin/python3 -m pip install [options] [-e] <local project path> ...
  /usr/local/bin/python3 -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [34]:
import plotly.io as pio

# Plot forecast for each state
for state in states:
    forecast = forecasts[state]
    fig = plot_plotly(model, forecast)
    fig.update_layout(
        title=f"Forecast for {state}",
        xaxis_title="Date",
        yaxis_title="Demand (MW)",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    # Save the plot as an HTML file
    file_name = f"forecast_{state}.html"
    pio.write_html(fig, file_name)
    print(f"Forecast plot for {state} saved as {file_name}")


Forecast plot for DD saved as forecast_DD.html
Forecast plot for Chattishgarh saved as forecast_Chattishgarh.html
Forecast plot for Maharashtra saved as forecast_Maharashtra.html
Forecast plot for Goa saved as forecast_Goa.html
Forecast plot for Gujrat saved as forecast_Gujrat.html
Forecast plot for Madhya Pradesh saved as forecast_Madhya Pradesh.html
Forecast plot for DNH saved as forecast_DNH.html


In [14]:
data[data['state'] == 'Maharashtra']

,ds,state,frequency,demand
3,2021-01-01 01:15:00,Maharashtra,50.0184615384616,18781.0427350427
9,2021-01-01 02:15:00,Maharashtra,50.0233333333334,18487.2666666667
15,2021-01-01 03:15:00,Maharashtra,50.0281666666667,18415.1416666667
23,2021-01-01 04:15:00,Maharashtra,50.0045378151261,18148.1932773109
28,2021-01-01 05:15:00,Maharashtra,50.0190833333334,18227.55
...,...,...,...,...
69111,2023-05-13 08:15:00,Maharashtra,50.0075,23692.25
69119,2023-05-13 09:15:00,Maharashtra,50.0173333333333,24620.5666666667
69128,2023-05-13 10:15:00,Maharashtra,49.9875,25184.5333333333
69134,2023-05-13 11:15:00,Maharashtra,49.9393333333334,25573.1
